In [1]:
import json

import datetime, time
import simpy

import shapely.geometry
from simplekml import Kml, Style

import pandas as pd
import openclsim.core as core
import openclsim.model as model

import datetime

import numpy as np

from plot import vessel_planning

from vo_colos.utils.object_registry import SiteRegistry, VesselRegistry
from vo_colos.utils.object_registry import ClassInspect

from vo_colos.utils.base_task import SimulationFactory

In [11]:
with open("./config.json", 'r') as fp:
    config = json.load(fp)

In [12]:
print(json.dumps(config, indent=4))

{
    "tag": [
        "Cutter",
        "Barge",
        "example",
        "demo"
    ],
    "engine": "colos",
    "config": {
        "simulationProperties": {},
        "activities": [
            {
                "name": "SingeRunCycle1",
                "class": "WhileActivity",
                "properties": {
                    "sub_process": {
                        "name": "WhileFull1",
                        "class": "SequenceActivity",
                        "properties": {
                            "sub_processes": [
                                {
                                    "name": "sailing_empty1",
                                    "class": "MoveActivity",
                                    "properties": {
                                        "mover": "barge_1",
                                        "destination": "from_site"
                                    }
                                },
                                {
              

In [4]:
class message:
    id = 1
    task = {
        "id": 1,
        "engine": "colos",
        "project_id": "1234567890",
        "tag": ["testsimulation"],
        "config": config["config"],
    }
    status = "new"

sim = SimulationFactory(message)

In [5]:
sim.simulate()

init
level: 55
completed init
init
level: 55
completed init
init
level: 0
completed init
init
level: 0
completed init
while Activity keep_resources []
start put_available
register new event 
get_full_event : default
start get_available
evaluate event <Event() object at 0x1d563b08c48> as triggered False and processed False
evaluate event <Event() object at 0x1d563b08cc8> as triggered False and processed False
start event instance None
while Activity keep_resources []
start put_available
register new event 
get_full_event : default
start get_available
evaluate event <Event() object at 0x1d563b13208> as triggered False and processed False
evaluate event <Event() object at 0x1d563b08cc8> as triggered False and processed False
start event expression [{'type': 'activity', 'state': 'done', 'name': 'SingeRunCycle_1'}]
start event <Process(conditional_process) object at 0x1d563b08988>
start event instance <Process(conditional_process) object at 0x1d563b08988>
conditional 
start event instance N

released origin : {}
released processor : {}
{}
Activity end(SingeRunCycle_1 loading)
keep_resources []
start event instance None
Mover_move before mover resource request
Mover_move after mover resource request
Mover_move after move
Activity end(SingeRunCycle_1 sailing filled)
keep_resources []
SHift amount Activity keep_resources []
start event instance None
start : {}
{'origin.86b1241c-acae-11ea-8135-2079183a5c9a': 5, 'destination.to_site': 999970}
destination request : {<simpy.resources.resource.Resource object at 0x000001D563B08508>: <Request() object at 0x1d563b08dc8>}
shift amount process keep_resources []
start get_available
processor request : {<simpy.resources.resource.Resource object at 0x000001D563B08508>: <Request() object at 0x1d563b08dc8>, <simpy.resources.resource.Resource object at 0x000001D563AFF4C8>: <Request() object at 0x1d563b135c8>}
site request : {<simpy.resources.resource.Resource object at 0x000001D563B08508>: <Request() object at 0x1d563b08dc8>, <simpy.resourc

{'origin.86b1241c-acae-11ea-8135-2079183a5c9a': 5, 'destination.to_site': 999935}
destination request : {<simpy.resources.resource.Resource object at 0x000001D563B08508>: <Request() object at 0x1d563b028c8>}
shift amount process keep_resources []
start get_available
processor request : {<simpy.resources.resource.Resource object at 0x000001D563B08508>: <Request() object at 0x1d563b028c8>, <simpy.resources.resource.Resource object at 0x000001D563AFF4C8>: <Request() object at 0x1d563b08688>}
site request : {<simpy.resources.resource.Resource object at 0x000001D563B08508>: <Request() object at 0x1d563b028c8>, <simpy.resources.resource.Resource object at 0x000001D563AFF4C8>: <Request() object at 0x1d563b08688>}
end requestIfAvailable : {<simpy.resources.resource.Resource object at 0x000001D563B08508>: <Request() object at 0x1d563b028c8>, <simpy.resources.resource.Resource object at 0x000001D563AFF4C8>: <Request() object at 0x1d563b08688>}
after req resource if available : {<simpy.resources.

Mover_move after move
Activity end(SingeRunCycle_2 sailing empty)
keep_resources []
SHift amount Activity keep_resources []
start event instance None
start : {}
{'origin.from_site_2': 30, 'destination.86b1241c-acae-11ea-8135-2079183a5c9a': 5}
destination request : {<simpy.resources.resource.Resource object at 0x000001D563AFF4C8>: <Request() object at 0x1d563af4f48>}
shift amount process keep_resources []
start get_available
processor request : {<simpy.resources.resource.Resource object at 0x000001D563AFF4C8>: <Request() object at 0x1d563af4f48>}
site request : {<simpy.resources.resource.Resource object at 0x000001D563AFF4C8>: <Request() object at 0x1d563af4f48>, <simpy.resources.resource.Resource object at 0x000001D563B08248>: <Request() object at 0x1d563b3b2c8>}
end requestIfAvailable : {<simpy.resources.resource.Resource object at 0x000001D563AFF4C8>: <Request() object at 0x1d563af4f48>, <simpy.resources.resource.Resource object at 0x000001D563B08248>: <Request() object at 0x1d563b3b

In [6]:
objects = list(sim.vessels)
objects.extend(sim.locations)


activities = []
for obj in objects:
    activities.extend(set(obj.log["Message"]))

activities = list(set(activities))

vessel_planning(objects, activities)

In [7]:
[a.name for a in sim.activities]

['SingeRunCycle_1', 'SingeRunCycle_2']

In [8]:
sim.registry

{'name': {'SingeRunCycle_1 sailing empty': [<openclsim.model.MoveActivity at 0x1d563af4988>],
  'SingeRunCycle_1 loading': [<openclsim.model.ShiftAmountActivity at 0x1d55d18bc08>],
  'SingeRunCycle_1 sailing filled': [<openclsim.model.MoveActivity at 0x1d5630d3488>],
  'SingeRunCycle_1 unloading': [<openclsim.model.ShiftAmountActivity at 0x1d55a3875c8>],
  'SingeRunCycle_1 sequence': [<openclsim.model.SequentialActivity at 0x1d55a387388>],
  'SingeRunCycle_1': [<openclsim.model.WhileActivity at 0x1d55c868448>],
  'SingeRunCycle_2 sailing empty': [<openclsim.model.MoveActivity at 0x1d563b08ec8>],
  'SingeRunCycle_2 loading': [<openclsim.model.ShiftAmountActivity at 0x1d563b12188>],
  'SingeRunCycle_2 sailing filled': [<openclsim.model.MoveActivity at 0x1d563b12488>],
  'SingeRunCycle_2 unloading': [<openclsim.model.ShiftAmountActivity at 0x1d563b12788>],
  'SingeRunCycle_2 sequence': [<openclsim.model.SequentialActivity at 0x1d563b12ac8>],
  'SingeRunCycle_2': [<openclsim.model.WhileAct

In [9]:
logs = sim.get_logs()

In [10]:
np.int32

numpy.int32